# Pytorch Lightning Estimator interface for TradeBot transformer 



In [1]:
import sys
sys.path.append('/Users/nhassen/Documents/ProjectQuant/MyRepos/test/GPT-NeoX-for-Time-Series-Forecasting/')

In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import pytorch_lightning as pl
from itertools import islice
from GPT_NEO.lightning_module import TradeBotLightning
from GPT_NEO.estimator import TradeBotLightEstimator
from GPT_NEO.estimator import TradeBotLightning

/Users/nhassen/opt/anaconda3/envs/env_LLMTS/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [3]:
import torch
from gluon.dataset import generate_backtesting_datasets
from gluon.metrics import compute_validation_metrics
from gluon.plots import plot_four_forecasts
from gluonts.evaluation.backtest import make_evaluation_predictions, Evaluator

In [4]:
history_factor = 3
backtest_id = 2

metadata, train_data, test_data = generate_backtesting_datasets("electricity_hourly", backtest_id, history_factor, use_cached=False)

Download electricity_hourly_dataset.zip:: 11.3MB [00:04, 2.79MB/s]
creating json files: 100%|██████████| 321/321 [00:00<00:00, 309866.88it/s]


In [5]:
for entry in list(train_data):
    entry["target"] = entry["target"][:24, :]
for entry in list(test_data):
    entry["target"] = entry["target"][:24, :]
entry

{'target': array([[   0.,    0.,    0., ...,   11.,   11.,   12.],
        [   0.,    0.,    0., ...,   97.,   88.,   82.],
        [   0.,    0.,    0., ...,    8.,    8.,    8.],
        ...,
        [   0.,    0.,    0., ..., 1437., 1404., 1134.],
        [   0.,    0.,    0., ...,  124.,  109.,  116.],
        [   0.,    0.,    0., ...,  178.,  151.,  139.]], dtype=float32),
 'start': Period('2014-11-28 00:00', 'H'),
 'feat_static_cat': array([0], dtype=int32)}

In [6]:
#Original Energy dataset
train_tf_entry = next(iter(list(train_data)))
[k for k in train_tf_entry.keys()]

['target', 'start', 'feat_static_cat']

In [7]:
list(train_data)[0]

{'target': array([[  14.,   18.,   21., ...,   11.,   12.,   10.],
        [  69.,   92.,   96., ...,  100.,   94.,   86.],
        [ 234.,  312.,  312., ...,    8.,    8.,    8.],
        ...,
        [ 885., 1074.,  936., ..., 1301., 1193.,  979.],
        [ 122.,  150.,  149., ...,  126.,  118.,  113.],
        [ 102.,  127.,  117., ...,  142.,  129.,  106.]], dtype=float32),
 'start': Period('2012-01-01 00:00', 'H'),
 'feat_static_cat': array([0], dtype=int32)}

Create the Lightning version of the TradeBOT model.

The model parameters are almost all in the `model_parameters` dictionary.

In [9]:

net_estimator = TradeBotLightEstimator(
    num_samples = 100,
    model_parameters= {
        "gamma":0.8,
        "l_norm": 2,
        "data_normalization":"standardization",
        "loss_normalization":"series",
        "series_embedding_dim":13,
        "input_encoder_layers":3,
        "input_encoding_normalization":True,
        "encoder": {
            "attention_layers":3,
            "attention_heads": 3,
            "attention_dim": 4,
            "attention_feedforward_dim": 12,
        },
        "quantile_decoder":{
             "min_u": 0.01,
             "max_u": 0.99,
            "attentional_quantile": {
                "attention_heads": 3,
                "attention_layers": 3,
                "attention_dim": 12,
                "mlp_layers": 3,
                "mlp_dim": 16,
                "resolution": 50,
            },
        }
    },
    learning_rate = 1e-3,
    trainer_kwargs=dict(max_epochs=3, accelerator="cpu"),
    num_series = list(train_data)[0]["target"].shape[0],
    history_length = history_factor * metadata.prediction_length,
    prediction_length = metadata.prediction_length,
    freq = metadata.freq,
    cdf_normalization = True,
    num_parallel_samples = 100,
    batch_size = 32,
    num_batches_per_epoch = 4,
)

Train the model. Lightning automatically send the model to GPU if the accelerator is set accordingly, and send it back to CPU after training.

The tuner can automatically find the maximum batch size which can fit in memory, but we give it a maximum number of trials since it does not stop as it goes above the number of samples per epoch.

In [10]:
predictor = net_estimator.train(train_data)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/nhassen/opt/anaconda3/envs/env_LLMTS/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
/Users/nhassen/opt/anaconda3/envs/env_LLMTS/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:72: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name | T

Training: 0it [00:00, ?it/s]

/Users/nhassen/opt/anaconda3/envs/env_LLMTS/lib/python3.9/site-packages/gluonts/transform/convert.py:566: RuntimeWarning: divide by zero encountered in divide
  x_diff == 0.0, np.zeros_like(x_diff), y_diff / x_diff
Epoch 0, global step 30: 'train_loss' reached -5.59897 (best -5.59897), saving model to '/Users/nhassen/Documents/ProjectQuant/MyRepos/test/GPT-NeoX-for-Time-Series-Forecasting/lightning_logs/version_37/checkpoints/epoch=0-step=30.ckpt' as top 1
/Users/nhassen/opt/anaconda3/envs/env_LLMTS/lib/python3.9/site-packages/pytorch_lightning/trainer/call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_data, predictor=predictor
)

In [ ]:
forecasts = list(forecast_it)
tss = list(ts_it)

In [ ]:
evaluator = Evaluator()
agg_metrics, ts_metrics = evaluator(
    iter(tss), iter(forecasts), num_series=len(test_data)
)

In [ ]:
agg_metrics

In [ ]:
metrics = compute_validation_metrics(
    predictor=predictor,
    dataset=test_data,
    window_length=net_estimator.history_length + net_estimator.prediction_length,
    num_samples=100,
    split=False,
)

In [ ]:
metrics

In [ ]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_data, predictor=predictor, num_samples=100
)
forecasts = list(forecast_it)
targets = list(ts_it)

In [ ]:
plot_four_forecasts(
    forecasts=forecasts,
    targets=targets,
    selection=[(0, 0), (1, 5), (2, 10), (3, 15)],
    tick_freq="day",
    history_length=net_estimator.history_length,
)